<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/Unet%2B%2Bfull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unet plus 3 è promettente ma da evitare per ora

Direi di applicare ++ ma con una strategia da cecchino; si congelano encoder(addestrato) e si passa solo il decoder che deve creare le maschere... sotto

| Mossa                | Codice                                                       | Riduzione stimata                                            | Perdita di accuracy                                         |
| -------------------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ----------------------------------------------------------- |
| **AMP** (già attivo) | `with torch.cuda.amp.autocast()`                             | ‑35 % activations ([PyTorch Forums][1], [PyTorch Forums][2]) | nessuna                                                     |
| **Freeze encoder**   | `for p in model.encoder.parameters(): p.requires_grad=False` | ‑25‑30 % (niente gradienti) ([PyTorch Forums][3])            | trascurabile su dataset piccoli                             |
| **Encoder leggero**  | `encoder_name="efficientnet-b0"`                             | ‑35‑40 % VRAM ([developer.furiosa.ai][4])                    | 0 – 1 pt Dice grazie ai pesi ImageNet ([docs.vultr.com][5]) |

[1]: https://discuss.pytorch.org/t/explanation-of-exact-effect-of-amp-on-memory-usage/131358?utm_source=chatgpt.com "Explanation of exact effect of AMP on memory usage - mixed-precision"
[2]: https://discuss.pytorch.org/t/increased-memory-usage-with-amp/125486?utm_source=chatgpt.com "Increased memory usage with AMP - PyTorch Forums"
[3]: https://discuss.pytorch.org/t/does-freezzing-part-of-the-network-saves-gpu-memory/107781?utm_source=chatgpt.com "Does freezzing part of the network saves GPU memory?"
[4]: https://developer.furiosa.ai/furiosa-models/latest/models/efficientnet_b0/?utm_source=chatgpt.com "EfficientNetB0 - Furiosa Models"
[5]: https://docs.vultr.com/how-to-use-gradient-accumulation-to-overcome-gpu-memory-limitations?utm_source=chatgpt.com "How to use Gradient Accumulation to Overcome GPU Memory ..."


In [ ]:
# ===================================================================
# CELLA 1: SETUP E CARICAMENTO DATI 512px
# ===================================================================
print("--- [1/4] Setup e Caricamento Dati 512px ---")
!pip install -q segmentation-models-pytorch==0.3.3 albumentations torchinfo
import torch, cv2, json, pathlib, numpy as np
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.notebook import tqdm

print("✅ Librerie pronte.")

# Percorso del dataset GIÀ RIDIMENSIONATO a 512px su Drive
DATA_512_DRIVE_PATH = "/content/drive/MyDrive/Colab_Datasets/Augmented_Dataset_55" # Assumo sia questo
LOCAL_DATA_PATH = pathlib.Path("/content/dataset_512")

if not LOCAL_DATA_PATH.exists():
    print(f"Copiando dati da '{DATA_512_DRIVE_PATH}'...")
    !cp -r "{DATA_512_DRIVE_PATH}" "{LOCAL_DATA_PATH}"
    print("✅ Copia completata.")
else:
    print(f"ℹ️ Dataset già presente in '{LOCAL_DATA_PATH}'.")

In [ ]:
#### Cella 2: Dataset e DataLoaders
# ===================================================================
# CELLA 2: DATASET E DATALOADERS
# ===================================================================
print("--- [2/4] Definizione Dataset e Creazione DataLoaders ---")

class SegmentationDataset(Dataset):
    def __init__(self, ids, img_dir, msk_dir):
        self.ids, self.img_dir, self.msk_dir = ids, img_dir, msk_dir
    def __len__(self): return len(self.ids)
    def __getitem__(self, i):
        stem = self.ids[i]
        img = cv2.imread(str(self.img_dir / f"{stem}.png"))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        msk = cv2.imread(str(self.msk_dir / f"{stem}.png"), cv2.IMREAD_UNCHANGED)
        img_tensor = torch.from_numpy(img/255.0).permute(2, 0, 1).float()
        msk_tensor = torch.from_numpy(msk).long()
        return img_tensor, msk_tensor

split_path = LOCAL_DATA_PATH / "split.json"
with open(split_path, 'r') as f: splits = json.load(f)
train_ids, val_ids = splits['train'], splits['val']

IMG_DIR = LOCAL_DATA_PATH / "images"
MSK_DIR = LOCAL_DATA_PATH / "masks"

# Batch size che DOVREBBERO funzionare ora
# --- Crea i Dataset ---
train_ds = SegmentationDataset(train_ids, IMG_DIR, MSK_DIR)
val_ds = SegmentationDataset(val_ids, IMG_DIR, MSK_DIR)
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 4
#potresti aumentare workers senza effettivo aumento di vram = 4?
train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=2)
val_dl = DataLoader(val_ds, batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=2)

print(f"✅ DataLoaders pronti. Training set: {len(train_ds)}.")

#testiamo la B0 per vantaggi su vram con minima perdita dice su piccoli dataset; B1 forse sarebbe meglio ma per ora ci accontetntiamo

In [ ]:
#### Cella 3: Modello con Encoder Congelato e Loss
# ===================================================================
# CELLA 3: MODELLO CON ENCODER CONGELATO E LOSS
# ===================================================================
print("--- [3/4] Definizione Modello e Componenti di Training ---")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# --- La nostra Focal Loss custom ---
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha, self.gamma, self.reduction = alpha, gamma, reduction
        self.crit = nn.CrossEntropyLoss(weight=None, reduction='none')
    def forward(self, output, target):
        ce_loss = self.crit(output, target)
        pt = torch.exp(-ce_loss)
        loss = ( (1 - pt)**self.gamma ) * ce_loss
        if self.alpha is not None: loss = self.alpha[target] * loss
        if self.reduction == 'mean': return loss.mean()
        return loss

# --- PARAMETRI E MODELLO ---
LEARNING_RATE, NUM_EPOCHS, N_CLASSES = 1e-4, 50, 3
#cambia encoder backbone per ottimizzare ram
model = smp.UnetPlusPlus(encoder_name="efficientnet-b0", encoder_weights="imagenet", in_channels=3, classes=N_CLASSES).to(DEVICE)
#model = smp.UnetPlusPlus(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=N_CLASSES).to(DEVICE)

# --- LA MOSSA VINCENTE: CONGELARE L'ENCODER ---
for param in model.encoder.parameters():
    param.requires_grad = False
print("✅ Encoder CONGELATO. Verrà addestrato solo il decoder.")

# Filtra i parametri per l'optimizer: solo quelli che richiedono gradienti
trainable_params = [p for p in model.parameters() if p.requires_grad]

# --- LOSS, OPTIMIZER, SCHEDULER ---
class_weights = torch.tensor([0.25, 0.5, 1.0]).to(DEVICE)
loss_fn = FocalLoss(alpha=class_weights, gamma=2.0)
optimizer = torch.optim.AdamW(trainable_params, lr=LEARNING_RATE) # Passa solo i parametri addestrabili
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

print(f"✅ Modello: UnetPlusPlus-B0 (frozen), Loss: FocalLoss (Custom). Pronti.")

In [ ]:
# ===================================================================
# CELLA 4 (ex 4.9): PANNELLO DI CONTROLLO PRE-TRAINING CON CHECK MEMORIA
# ===================================================================
print("--- [4/5] CONTROLLO FINALE PRIMA DEL TRAINING ---")

def mem_report(msg=""):
    if DEVICE == 'cuda':
        res = torch.cuda.memory_reserved()/1e9
        alloc = torch.cuda.memory_allocated()/1e9
        print(f"{msg:>25}: reserved={res:.2f} GB, allocated={alloc:.2f} GB")

# --- 1. Controlla il DataLoader ---
try:
    images, masks = next(iter(train_dl))
    print("✅ DataLoader: OK")
    print(f"    -> Dimensione batch immagini: {images.shape}")
    print(f"    -> Dimensione batch maschere: {masks.shape}")
except Exception as e:
    print(f"❌ ERRORE NEL DATALOADER: {e}")
    raise

# --- 2. Controlla il Modello e la Memoria ---
model.eval()
if DEVICE == 'cuda': torch.cuda.empty_cache() # Pulisci la cache prima del test
mem_report("Inizio Test Modello")

with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
    try:
        test_images = images.to(DEVICE)
        outputs = model(test_images)
        mem_report("Dopo Forward Pass") # VEDIAMO QUANTO CONSUMA LA FORWARD
        print("✅ Modello: OK")
        print(f"    -> Dimensione output modello: {outputs.shape}")
    except Exception as e:
        print(f"❌ ERRORE NEL MODELLO: {e}")
        raise
model.train()

# --- 3. Controlla la Loss Function ---
try:
    test_masks = masks.to(DEVICE)
    loss = loss_fn(outputs, test_masks)
    print("✅ Loss Function: OK")
    print(f"    -> Tipo di Loss: {loss_fn.__class__.__name__}")
    print(f"    -> Valore di loss di test: {loss.item()}")
except Exception as e:
    print(f"❌ ERRORE NELLA LOSS FUNCTION: {e}")
    print("\n--- DEBUG DIMENSIONI ---")
    print(f"Shape Output (input): {outputs.shape}")
    print(f"Shape Maschere (target): {masks.shape}")
    raise

# Pulisci la memoria occupata dai tensori di test
del images, masks, test_images, test_masks, outputs, loss
if DEVICE == 'cuda': torch.cuda.empty_cache()
mem_report("Dopo Pulizia Test")

print("\n🚀 CONTROLLI SUPERATI. PRONTI PER IL TRAINING. 🚀")

In [ ]:
#### Cella 5: Training Loop con AMP (come richiesto)

#Questa è la cella finale. Pulita, con la mixed precision (AMP) per massimizzare il risparmio di memoria.
#
#```python
# ===================================================================
# CELLA 5: TRAINING LOOP
# ===================================================================
print("--- [5/5] Inizio Training ---")

# Inizializza lo scaler per la mixed precision (solo se si usa la GPU)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == 'cuda'))
best_val_loss = float('inf')
checkpoint_path_B0 = "/content/drive/MyDrive/Pesi/PesiUNET/best_model_FrozenEncoder_Epochs50_GPU_B0_v2.pth"

# Pulisci la cache della GPU un'ultima volta prima di iniziare
if DEVICE == 'cuda': torch.cuda.empty_cache()

for epoch in range(NUM_EPOCHS):
    # --- Training Loop ---
    model.train()
    train_loss = 0
    loop_train = tqdm(train_dl, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]", leave=False)
    for images, masks in loop_train:
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)

        # Usa il contesto autocast per la mixed precision
        with torch.cuda.amp.autocast(enabled=(DEVICE == 'cuda')):
            outputs = model(images)
            loss = loss_fn(outputs, masks)

        # Scala la loss e fa la backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    # --- Validation Loop ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in val_dl:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            # Usa autocast anche in validazione per coerenza e velocità
            with torch.cuda.amp.autocast(enabled=(DEVICE == 'cuda')):
                outputs = model(images)
                loss = loss_fn(outputs, masks)
            val_loss += loss.item()

    avg_train_loss = train_loss / len(train_dl)
    avg_val_loss = val_loss / len(val_dl)
    current_lr = scheduler.get_last_lr()[0]

    print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, LR: {current_lr:.6f}")

    scheduler.step()

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), checkpoint_path_B0)
        print(f"--> New best model saved to {checkpoint_path_B0} with val loss {best_val_loss:.4f}")

print("\n--- Training Completato ---")

In [ ]:
# ===================================================================
# CELLA 6: INFERENZA TOTALE E SALVATAGGIO SU DISCO
# ===================================================================
print("--- [6/7] INFERENZA TOTALE SU TUTTO IL DATASET E SALVATAGGIO ---")

# --- 1. PARAMETRI ---
CHECKPOINT_PATH = "/content/drive/MyDrive/Pesi/PesiUNET/best_model_FrozenEncoder_Epochs50_GPU_B0_v2.pth"
# Il percorso dei dati 512px che hai usato per il training
DATA_PATH = pathlib.Path("/content/dataset_512")
OUTPUT_DIR = pathlib.Path("/content/inference_results_FULL")
OUTPUT_DIR.mkdir(exist_ok=True)
ENCODER_NAME = "efficientnet-b0" # Assicurati che sia quello giusto!

# --- 2. CARICA MODELLO ---
INFERENCE_MODEL = smp.UnetPlusPlus(
    encoder_name=ENCODER_NAME,
    encoder_weights=None,
    in_channels=3,
    classes=3
).to(DEVICE)
try:
    INFERENCE_MODEL.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))
    print(f"✅ Pesi caricati con successo da: {CHECKPOINT_PATH}")
except Exception as e:
    print(f"❌ ERRORE NEL CARICAMENTO DEI PESI: {e}")
    raise e
INFERENCE_MODEL.eval()

# --- 3. CARICA TUTTI GLI ID DELLE IMMAGINI ---
# Invece di usare solo il test set, prendiamo TUTTE le immagini per avere una visione completa
all_image_paths = sorted(list((DATA_PATH / "images").glob("*.png")))
all_stems = [p.stem for p in all_image_paths]
print(f"Trovate {len(all_stems)} immagini totali da processare.")

# --- 4. PRE-PROCESSING PER L'INFERENZA ---
inference_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# --- 5. LOOP DI INFERENZA SU TUTTO ---
print("Avvio del processo di inferenza totale...")
with torch.no_grad():
    for stem in tqdm(all_stems, desc="Processing All Images"):
        img_path = DATA_PATH / "images" / f"{stem}.png"
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        processed = inference_transform(image=image)
        image_normalized = processed['image']

        input_tensor = torch.from_numpy(image_normalized).permute(2,0,1).unsqueeze(0).to(DEVICE)

        with torch.amp.autocast(device_type=DEVICE, enabled=(DEVICE=='cuda')):
            output = INFERENCE_MODEL(input_tensor)

        predicted_mask = torch.argmax(output.squeeze(), dim=0).cpu().numpy().astype(np.uint8)

        save_path = OUTPUT_DIR / f"{stem}_pred.png"
        cv2.imwrite(str(save_path), predicted_mask)

print(f"\n✅ Inferenza completata. {len(all_stems)} maschere predette sono state salvate in '{OUTPUT_DIR}'.")

In [ ]:
#### Cella 7: Il Visualizzatore di Maschere

#Questa cella contiene la tua logica di visualizzazione, ma trasformata in una funzione potente e riutilizzabile. Esegue la visualizzazione a 3 pannelli (Originale, Reale, Prevista) che è il modo migliore per giudicare.

# ===================================================================
# CELLA 7: VISUALIZZATORE DI RISULTATI
# ===================================================================
print("--- [7/7] Visualizzatore di Risultati di Inferenza ---")

def visualize_prediction(stem_name):
    """
    Carica e visualizza l'immagine originale, la maschera reale e la maschera predetta
    per un dato 'stem' (nome del file).
    """
    original_img_path = DATA_PATH / "images" / f"{stem_name}.png"
    true_mask_path = DATA_PATH / "masks" / f"{stem_name}.png"
    pred_mask_path = OUTPUT_DIR / f"{stem_name}_pred.png"

    if not original_img_path.exists() or not pred_mask_path.exists():
        print(f"❌ Immagine o maschera predetta non trovata per lo stem: {stem_name}")
        return

    print(f"\n--- Risultati per: {stem_name} ---")

    original_img = cv2.imread(str(original_img_path))
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

    predicted_mask = cv2.imread(str(pred_mask_path), cv2.IMREAD_UNCHANGED)

    fig, axes = plt.subplots(1, 3, figsize=(20, 7))

    axes[0].imshow(original_img)
    axes[0].set_title("Immagine Originale")

    if true_mask_path.exists():
        true_mask = cv2.imread(str(true_mask_path), cv2.IMREAD_UNCHANGED)
        axes[1].imshow(true_mask, cmap='jet', vmin=0, vmax=2)
        axes[1].set_title("Maschera Reale")
    else:
        axes[1].set_title("Maschera Reale (Non trovata)")

    axes[2].imshow(predicted_mask, cmap='jet', vmin=0, vmax=2)
    axes[2].set_title("Maschera Prevista")

    for ax in axes:
        ax.axis('off')

    plt.show()

# --- ESEMPI DI UTILIZZO ---
# Ora puoi visualizzare qualsiasi immagine che vuoi, semplicemente chiamando la funzione.

# Visualizza un'immagine specifica
visualize_prediction("hole_0021_orig")

# Visualizza una delle immagini di test
if 'test_ids' in locals() and test_ids:
    visualize_prediction(test_ids[0])

# Visualizza un'altra immagine a caso
visualize_prediction("hole_0035_aug_5")

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

print(sorted(list(smp.encoders.encoders.keys()))[:30], "...")  # native
# timm bridge
print([k for k in smp.encoders.encoders.keys() if k.startswith("tu-")][:30])


In [ ]:
#@title Cella DEGENERE test consumo picco test di ogni backbone
from torch.amp import autocast, GradScaler
encoders_to_test = ["efficientnet-b0",
                    "efficientnet-b1",
                    "tu-mobilenetv3_large_100",
                    "resnet18","efficientnet-b7"]

for enc in encoders_to_test:
    model = smp.UnetPlusPlus(enc, encoder_weights="imagenet", in_channels=3, classes=3).cuda()
    with torch.no_grad(), autocast("cuda"):
        x = torch.randn(1,3,512,512, device="cuda")
        _ = model(x)
    print(enc, torch.cuda.max_memory_reserved()/1e9, "GB")
    torch.cuda.reset_peak_memory_stats(); torch.cuda.empty_cache()


#scelta backbone della rete
| Famiglia                                                | Parametri ↑                  | Peak VRAM\* (512², batch 2, fp32) | Accuracy tipica su set piccoli                     | Quando usarla                             |
| ------------------------------------------------------- | ---------------------------- | --------------------------------- | -------------------------------------------------- | ----------------------------------------- |
| **ResNet‑34** (baseline)                                | 21 M ([docs.pytorch.org][1]) | ≈ 11‑12 GiB                       | solida, ma pesante                                 | valore di partenza                        |
| **ResNet‑18**                                           | 11 M                         | ≈ 9 GiB                           | ‑1 pt Dice rispetto a R‑34                         | se vuoi cambiare il minimo indispensabile |
| **EfficientNet‑B0**                                     | **5.3 M** ([viso.ai][2])     | **≈ 7 GiB**                       | = / +0‑1 pt Dice (grazie allo scaling compresso)   | scelta più equilibrata                    |
| **EfficientNet‑B1**                                     | 7.9 M                        | ≈ 8 GiB                           | +0‑1 pt vs B0, ancora leggera                      | se hai 1 GiB in più di margine            |
| **MobileNet V3‑Large**                                  | 5.4 M ([keras.io][3])        | ≈ 6.5 GiB                         | ‑1‑3 pt Dice in texture complesse                  | massima velocità / min‑VRAM               |
| **HarDNet‑68**                                          | 17 M                         | ≈ 8‑9 GiB                         | +1 pt Dice nei polipi, buona velocità ([arXiv][4]) | se vuoi FPS senza perdere troppo          |
| \*stima UNet++ completo (decoder da 256→64), senza AMP. |                              |                                   |                                                    |                                           |

[1]: https://docs.pytorch.org/vision/0.21/models/generated/torchvision.models.resnet34.html?utm_source=chatgpt.com "resnet34 — Torchvision 0.21 documentation"
[2]: https://viso.ai/deep-learning/efficientnet/?utm_source=chatgpt.com "EfficientNet: Optimizing Deep Learning Efficiency - Viso Suite"
[3]: https://keras.io/api/applications/mobilenet/?utm_source=chatgpt.com "MobileNet, MobileNetV2, and MobileNetV3 - Keras"
[4]: https://arxiv.org/abs/2101.07172?utm_source=chatgpt.com "HarDNet-MSEG: A Simple Encoder-Decoder Polyp Segmentation ..."


In [ ]:
#3 · Come cambiano VRAM e tempo

#Il peso dei parametri incide poco; la VRAM è dominata dalle feature‑map prodotte dall’encoder:
#VRAM≈B  (∑lClHlWl)×bytes×F
#VRAM≈B(l∑​Cl​Hl​Wl​)×bytes×F
#
#dove C_l cala molto quando passi da ResNet a EfficientNet/MobileNet. Con batch 2, 512 px, fp16 (AMP) e UNet++ decoder standard:
#
#    ResNet‑34 ➜ ≈ 7‑8 GiB VRAM (con AMP).
#
#    EfficientNet‑B0 ➜ ≈ 4‑5 GiB (‑35 %).
#
#    MobileNet V3 ➜ ≈ 4 GiB (‑45 %).
#
#AMP dimezza i byte per activation (F≈2) e conviene sempre su T4

###ResNet‑34 ➜ ≈ 7‑8 GiB VRAM (con AMP).
###
###EfficientNet‑B0 ➜ ≈ 4‑5 GiB (‑35 %).
###
###MobileNet V3 ➜ ≈ 4 GiB (‑45 %).